In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
import glob


In [19]:
full_data = pd.read_csv("./prepro_data/full_data.csv")

### Warm UP

- Categorical 변수 만들기 
- Categorical 변수 label encoding

In [20]:
# Categorical variables
cat_already = ["bank_id", "product_id", "gender"]
cat = ["income_type", "employment_type", "houseown_type", "purpose"]
 
# Categorical로 만들어볼만한 변수
cont = ["loan_limit", "loan_rate", "birth_year", "credit_score", "yearly_income", "company_enter_month", "desired_amount" ,"existing_loan_cnt","existing_loan_amt"]
 
# 쓸모없을것 같은 변수 User ID, application_id는 반드시 삭제
del_var = ["loanapply_insert_time", "application_id", "insert_time", "user_id"]
 
# 타겟 변수
val_y = ["is_applied"]

In [21]:
# le_income_type

le_income_type = LabelEncoder()

le_income_fitted = le_income_type.fit_transform(full_data["income_type"])
income_inverted = le_income_type.inverse_transform(le_income_fitted)

print(le_income_type.classes_)

['EARNEDINCOME' 'EARNEDINCOME2' 'FREELANCER' 'OTHERINCOME' 'PRACTITIONER'
 'PRIVATEBUSINESS']


In [22]:
# le_employment_type

le_employment_type = LabelEncoder()

le_employment_fitted = le_employment_type.fit_transform(full_data["employment_type"])
employment_inverted = le_employment_type.inverse_transform(le_employment_fitted)

print(le_employment_type.classes_)

['계약직' '기타' '일용직' '정규직']


In [23]:
# le_houseown_type

le_houseown_type = LabelEncoder()

le_houseown_fitted = le_houseown_type.fit_transform(full_data["houseown_type"])
houseown_inverted = le_houseown_type.inverse_transform(le_houseown_fitted)

print(le_houseown_type.classes_)

['기타가족소유' '배우자' '자가' '전월세']


In [24]:
# le_purpose

le_purpose_type = LabelEncoder()

le_purpose_fitted = le_purpose_type.fit_transform(full_data["purpose"])
purpose_inverted = le_purpose_type.inverse_transform(le_purpose_fitted)

print(le_purpose_type.classes_)

['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']


In [25]:
#  여기에 추가 cont -> cat 추가

#### Categorical encoding 으로 컬럼 변환

In [26]:
full_data["income_type"]     = le_income_fitted
full_data["employment_type"] = le_employment_fitted
full_data["houseown_type"]   = le_houseown_fitted
full_data["purpose"]         = le_purpose_fitted

In [27]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4217519 entries, 0 to 4217518
Data columns (total 21 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   loanapply_insert_time  object 
 2   bank_id                int64  
 3   product_id             int64  
 4   loan_limit             float64
 5   loan_rate              float64
 6   is_applied             float64
 7   user_id                int64  
 8   birth_year             float64
 9   gender                 float64
 10  insert_time            object 
 11  credit_score           float64
 12  yearly_income          float64
 13  income_type            int64  
 14  company_enter_month    int64  
 15  employment_type        int64  
 16  houseown_type          int64  
 17  desired_amount         float64
 18  purpose                int64  
 19  existing_loan_cnt      float64
 20  existing_loan_amt      float64
dtypes: float64(10), int64(9), object(2)
memory usage: 675.

### Split 

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train, val = train_test_split(full_data, test_size=0.20,   random_state=20205289,  stratify=full_data['is_applied'])
train, test = train_test_split(train, test_size=0.125,   random_state=20205289,  stratify=train['is_applied'])

In [30]:
train.to_csv("./prepro_data/train.csv", index = False)
val.to_csv("./prepro_data/val.csv", index = False)
test.to_csv("./prepro_data/test.csv", index = False)


### Step 1. Feature eng

1. 새롭게 만든 파생변수가 논리적이어야 한다.
    - 1.1. Data driven
        - 만든 파생변수가 대출 승인 받았을때와 받지 않았을때의 distribution 차이. plot, SKlearn 
        - EDA 과정 다음 Link 확인 : https://dacon.io/competitions/official/235927/codeshare/6052?page=1&dtype=recent
        - EDA 과정(파생변수 만들기) : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent 
        
    - 1.2. Stat driven
        - 만든 파생변수가 통계적으로 is_applied(Y/N)에서 차이가 있는지 검정.
        - Numeric 한 변수면 -> 만든 파생변수가 정규분포를 따르는지 확인 샤피로 웰콘슨 테스트(정규분포 테스트) -> t-test
        - Numeric 하지 않으면 -> 비모수검정, 맨 흴트이 윌콕슨 테스트 등
        
2. 만든 파생변수 N개를 가지고 간단한 베이스라인 ML 모델로 확인 : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent
    - Step 1. 기본 변수로만 사용해서 성능확인 // 파생변수 N개만 사용해서 성능확인 -> 각각 2개확인
    - Step 2. 기본 변수로만 사용해서 성능확인 + 파생변수 1개씩 성능확인 (총 nCk 조합)
    
    
### Q1. Loan_limit, loan_rate가 중요한가?

- 대출이 승인되었는지 안되었는지를 예측하기 위해서 loan_limit & roan_rate가 중요한지 고민해보기
- "우선은 중요하지 않다" 라고 가정하고 결측치는 제거
- 대체적으로 is_applied가 1로 나타나는데 이는 label noise인가? 만약 label noise라면 확인할방법은?

In [ ]:
Randomforest
SVM